In [8]:
from dataclasses import dataclass
import numpy as np

def get_input(n):
    with open('input_'+n+'.txt', 'r') as infile:
        return infile.read().strip()


puzzle = get_input('17')

def parse_input(puzzle):
    lines = puzzle.strip().split('\n')
    lines = list(map(lambda x: list(map(int, x.__iter__())), lines))
    res = np.array(lines)
    return res




In [9]:
sample1 = """2413432311323
3215453535623
3255245654254
3446585845452
4546657867536
1438598798454
4457876987766
3637877979653
4654967986887
4564679986453
1224686865563
2546548887735
4322674655533"""


In [ ]:
from enum import Enum
from typing import Tuple


class CityGrid:
    def __init__(self, string):
        self.grid = parse_input(string)
        self.xmax = len(self.grid[0])-1    
        self.ymax = len(self.grid)-1
        
    def get_entry(self,x,y):
        return self.grid[y][x]
    
    def is_valid(self, x,y):
        return all([x>=0, y>=0, x<=self.xmax, y<=self.ymax])

@dataclass(frozen=True, order=True, repr=False)
class Direction(Enum):
    NORTH = (0,-1)
    SOUTH = (0,1)
    WEST = (-1,0)
    EAST = (1,0)

@dataclass(frozen=True, order=True)  
class Crucible:
    pos: Tuple[int]
    direction: Direction
    run: int = 0
    
    def advance(self):
        pos = tuple((a+b for a,b, in zip(self.pos , self.direction.value)))
        run = self.run + 1
        return Crucible(pos, self.direction, run)
    
    def turn(self, clockwise:bool):
        order = [
            Direction.NORTH,
            Direction.WEST,
            Direction.SOUTH,
            Direction.EAST
        ]
        idx = order.index(self.direction)
        idx_offset = -1 if clockwise else +1
        new_direction = order[(idx+idx_offset) % 4] 
        return Crucible(self.pos, new_direction, 0)  
                  
    
    def next_states(self):
            moves = [ self.turn(clockwise=True).advance(), self.turn(clockwise=False).advance() ]
            if self.run < 3:
                moves.append(self.advance())
            return moves
        
       

In [58]:
from queue import PriorityQueue


def shortest_path(puzzle):
    city = CityGrid(puzzle)
    queue = PriorityQueue()
    start = (0,0)
    goal = (city.xmax,city.ymax)
    min_heat_losses = dict()
    visited = []
    min_heat_losses[start] = 0
    queue.put((city.get_entry(*start), Crucible(start, Direction.SOUTH, 0)))

    while queue:
        heat_lost, current_state= queue.get()
        print(heat_lost, current_state)
        known_loss = min_heat_losses.get(current_state, float('inf'))  
        if heat_lost < known_loss:
            min_heat_losses[current_state] = heat_lost
        
        visited.append(current_state)
        if current_state.pos == goal:
            break
        
        for c in current_state.next_states():
            if (not city.is_valid(*c.pos)) or c in visited:
                continue
            queue.put((heat_lost+city.get_entry(*c.pos), c))
    
    for v in visited:
        print(v)
    return min_heat_losses[current_state]
             


assert Crucible((0,0), Direction.NORTH, 1).turn(clockwise=True) == Crucible((0,0), Direction.EAST, 0)
assert Crucible((0,0), Direction.NORTH, 1).turn(clockwise=False) == Crucible((0,0), Direction.WEST, 0)
assert Crucible((0,0), Direction.NORTH, 1).advance() == Crucible((0,-1), Direction.NORTH, 2)

print(Crucible((0,0), Direction.NORTH, 3).next_states())
#shortest_path(sample1)

shortest_path('''123
456
789''')

[Crucible(pos=(-1, 0), direction=<Direction.WEST: (-1, 0)>, run=1), Crucible(pos=(1, 0), direction=<Direction.EAST: (1, 0)>, run=1)]
1 Crucible(pos=(0, 0), direction=<Direction.SOUTH: (0, 1)>, run=0)
3 Crucible(pos=(1, 0), direction=<Direction.EAST: (1, 0)>, run=1)
4 Crucible(pos=(0, 0), direction=<Direction.WEST: (-1, 0)>, run=1)
5 Crucible(pos=(0, 1), direction=<Direction.SOUTH: (0, 1)>, run=1)
6 Crucible(pos=(2, 0), direction=<Direction.EAST: (1, 0)>, run=1)
6 Crucible(pos=(2, 0), direction=<Direction.EAST: (1, 0)>, run=2)
10 Crucible(pos=(1, 1), direction=<Direction.EAST: (1, 0)>, run=1)
12 Crucible(pos=(0, 2), direction=<Direction.SOUTH: (0, 1)>, run=2)
16 Crucible(pos=(2, 1), direction=<Direction.EAST: (1, 0)>, run=1)
16 Crucible(pos=(2, 1), direction=<Direction.EAST: (1, 0)>, run=2)
20 Crucible(pos=(1, 2), direction=<Direction.EAST: (1, 0)>, run=1)
27 Crucible(pos=(0, 2), direction=<Direction.WEST: (-1, 0)>, run=1)
29 Crucible(pos=(2, 2), direction=<Direction.EAST: (1, 0)>, run=

29